In [1]:
from langchain_community.document_loaders.text import TextLoader
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import numpy as np

In [2]:
def pretty_print_docs(docs):
    if isinstance(docs[0], tuple):
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}: ({d[1]})\n\n{d[0].page_content}\nMetadata: {d[0].metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )
    else:
        print(
            f"\n{'-' * 100}\n".join(
                [
                    f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                    for i, d in enumerate(docs)
                ]
            )
        )

def score_normalizer(val: float) -> float:
    return 1 - (1 / (1 + np.exp(val)))

K = 4

In [3]:
document_names = "nous_sommes_en_guerre.txt", "jeux_olympiques_paris.txt", "pacte_novation_linkedin.txt"

documents = [TextLoader(document_name, encoding="utf-8").load() for document_name in document_names]

# load an other document
# documents = TextLoader(

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = []
for document in documents:
    texts += text_splitter.split_documents(document)
for idx, text in enumerate(texts):
    text.metadata["id"] = idx
embedding = OllamaEmbeddings(model="mistral")

In [5]:
vectordb = FAISS.from_documents(texts, embedding, normalize_L2=True, distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE)
vectordb.save_local("local_vectorstore")

In [6]:
# vectordb = FAISS.load_local("local_vectorstore", embeddings=embedding, allow_dangerous_deserialization=True, distance_strategy=distance_strategy)

In [15]:
query = """c'est quoi le discours du président ?
"""

In [16]:
docs_proba = vectordb.similarity_search_with_score(query, k=30)
pretty_print_docs(docs_proba)

Document 1: (1.6569733619689941)

en athlétisme, une épreuve de 35 km marche mixte par équipe remplace le 50 km marche hommes en athlétisme ;
en canoë-kayak, deux épreuves de slalom extrême remplacent deux épreuves de sprint ;
en boxe, une catégorie féminine prend la place d'une catégorie masculine ;
en haltérophilie, quatre épreuves sont supprimées ;
en tir, le skeet par équipes mixtes remplace le trap par équipes mixtes ;
Metadata: {'source': 'jeux_olympiques_paris.txt', 'id': 72}
----------------------------------------------------------------------------------------------------
Document 2: (1.6600886583328247)

de nouveaux sports de leur choix, qui doivent ensuite être approuvés par le CIO. Pour Tokyo 2020, c'est le cas du karaté, du surf, du skateboard, de l'escalade sportive et du baseball/softball27. Par ailleurs, la présence de 28 sports ne définit pas à l'avance la nature ou le nombre des différentes épreuves, ni les éventuels ajouts proposés par le comité d'organisation, le p